In [2]:
# Loading packages
!pip install nltk

exec(open('utils.py').read())

# We fix the seeds to get consistent results
SEED = 1111
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
one.py is being run directly


In [3]:
from sklearn.model_selection import train_test_split

train_file = 'offenseval-training-v1.tsv'
train_corpus, train_labels = parse_dataset_task_c(train_file)
  
print(train_corpus)
print(train_labels)

print(len(train_corpus))
print(len(train_labels))

Offensives 1469
Non-offensives 2407
['user user go home youre drunk user maga trump2020  url', 'user liberals are all kookoo ', 'user was literally just talking about this laughing out loud all mass shootings like that have been set ups its propaganda used to divide us on major issues like gun control and terrorism', 'user canada doesnt need another cuck we already have enough looneyleft liberals fking up our great country qproofs trudeaumustgo', 'user you are a lying corrupt traitor nobody wants to hear anymore of your lies deepstatecorruption url', 'user user user gun control that is all these kids are asking for', 'user user user user laughing out loud   throwing the bullshit flag on such nonsense  putuporshutup   kavanaugh   maga   callthevotealready url', 'user user kind of like when conservatives wanna associate everyone to their left as communist antifa members', 'user user da fuck is going on people   theres the mens room and the womens room  pick one and stick w it ', 'user tb

In [0]:
valid_file = 'validation.tsv'
valid_corpus, valid_labels = parse_dataset_task_c(test_file)

train_corpus = train_corpus + valid_corpus
train_labels = train_labels + valid_labels

In [15]:
tokenized_corpus = tokenize_stemming(train_corpus)
print(len(tokenized_corpus))

3915


In [16]:
vocabulary = get_vocabulary(tokenized_corpus)
print(vocabulary)
print(len(vocabulary))

['user', 'go', 'home', 'your', 'drunk', 'maga', 'trump2020', '', 'url', 'liber', 'are', 'all', 'kookoo', 'wa', 'liter', 'just', 'talk', 'about', 'thi', 'laugh', 'out', 'loud', 'mass', 'shoot', 'like', 'that', 'have', 'been', 'set', 'up', 'it', 'propaganda', 'use', 'to', 'divid', 'us', 'on', 'major', 'issu', 'gun', 'control', 'and', 'terror', 'canada', 'doesnt', 'need', 'anoth', 'cuck', 'we', 'alreadi', 'enough', 'looneyleft', 'fking', 'our', 'great', 'countri', 'qproof', 'trudeaumustgo', 'you', 'a', 'lie', 'corrupt', 'traitor', 'nobodi', 'want', 'hear', 'anymor', 'of', 'deepstatecorrupt', 'is', 'these', 'kid', 'ask', 'for', 'throw', 'the', 'bullshit', 'flag', 'such', 'nonsens', 'putuporshutup', 'kavanaugh', 'callthevotealreadi', 'kind', 'when', 'conserv', 'wanna', 'associ', 'everyon', 'their', 'left', 'as', 'communist', 'antifa', 'member', 'da', 'fuck', 'peopl', 'there', 'men', 'room', 'women', 'pick', 'one', 'stick', 'w', 'tbh', 'day', 'i', 'dont', 'in', 'gener', 'connect', 'with', 'a

In [17]:
word2idx = get_word2idx(tokenized_corpus, vocabulary)
print(len(word2idx))

7831


In [18]:
idx2word = get_idx2word(vocabulary)
print(len(idx2word))

7830


In [19]:
sentences_lengths = [len(sentence) for sentence in tokenized_corpus]
max_len = np.max(np.array(sentences_lengths))

train_sentences_tensor, train_labels_tensor = parse_input(tokenized_corpus, word2idx, train_labels, max_len)

print(train_sentences_tensor)
print(train_labels_tensor)

print(train_sentences_tensor.shape)
print(train_labels_tensor.shape)

tensor([[   1,    1,    2,  ...,    0,    0,    0],
        [   1,   10,   11,  ...,    0,    0,    0],
        [   1,   14,   15,  ...,    0,    0,    0],
        ...,
        [7823, 1573,  975,  ...,    0,    0,    0],
        [7826, 7827,   97,  ...,    0,    0,    0],
        [ 718, 7829,    8,  ...,    0,    0,    0]])
tensor([0., 2., 1.,  ..., 0., 0., 2.])
torch.Size([3915, 102])
torch.Size([3915])


Offensives 9
Non-offensives 30
['hughhewitt mishacollins what the hell what does that tweet have to do with gun control maybe try to make your tweet have some sort of sense', 'guess ima have to beat some lil kids asses today cause lil becky not about to but my kid and get away with it httpstcorprmn2tban', 'breitbartnews little marco trump will get things done', 'realdonaldtrump everyone already know what you are trying to do with the fake news and witch hunt shit umits not working', 'if its not about me its about matt blah blah blah worrrry about your bf who makes you feel like shit but you act like yall good ', 'saeychung oh she is the worst ugh', 'breitbartnews obama fed the country shit sandwiches for 8 years maybe jim just has his addled mind confused about dates and who fed who what httpstcoynoqjq11lt', 'breitbartnews ironic because shes just like a shotguntwo cocks and shes ready to blow', 'hot usa latina girlfriend sucking so nicely 5 min httpstcogjuw6g7djx', 'realjack what you 

In [0]:
testing = read_csv('test_set_taskc.tsv')

tweets = [row['tweet'] for row in testing]
tokenized_corpus = tokenize_stemming(tweets)

test_tensor, labels_tensor = parse_input(tokenized_corpus, word2idx, test_labels, max_len)

In [48]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=30, max_depth=80, random_state=42)

clf.fit(train_sentences_tensor.cpu().numpy(), train_labels_tensor.cpu().numpy())
output = clf.predict(test_tensor.cpu().numpy())
print(np.count_nonzero(output))
print(output)

15
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [0]:
ids = [row['id'] for row in testing]
out = ['IND' if e == 0 else 'GRP' for e in output]
zipped = list(zip(ids, out))

with open('predictions_task_c_many_1.csv', "w") as f:
    writer = csv.writer(f, dialect='excel')
    for row in zipped:
        writer.writerow(row)